In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories for train and test data
train_dir = '/content/drive/MyDrive/Faulty_solar_panel'# Train
test_dir = '/content/drive/MyDrive/Solar_Test_data' # Test

# Image dimensions
img_height = 128
img_width = 128
batch_size = 32

# Create ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Rescale pixel values to [0, 1]
    rotation_range=20,   # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    horizontal_flip=True   # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale test images

# Load and preprocess train images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'  # Use 'sparse' for integer labels
)

# Load and preprocess test images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'  # Use 'sparse' for integer labels
)

# Get the number of classes
num_classes = len(train_generator.class_indices)

# Example of how to use the generators with model.fit()
# model.fit(train_generator, epochs=10, validation_data=test_generator)
print(num_classes)


Found 825 images belonging to 6 classes.
Found 60 images belonging to 6 classes.
6


In [ ]:
 # from google.colab import drive
# drive.mount('/content/drive')
# by Racksonsit Developers intern project
# Guide/AI Engineer:Abhay chougule

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# # Evaluate the model
# test_loss, test_acc = model.evaluate(test_generator)
# print('Test accuracy:', test_acc)


Epoch 1/10
26/26 [==============================] - 270s 10s/step - loss: 1.9614 - accuracy: 0.2255 - val_loss: 1.8529 - val_accuracy: 0.2333
Epoch 2/10
26/26 [==============================] - 43s 2s/step - loss: 1.6445 - accuracy: 0.3055 - val_loss: 1.6459 - val_accuracy: 0.3167
Epoch 3/10
26/26 [==============================] - 40s 2s/step - loss: 1.5900 - accuracy: 0.3733 - val_loss: 1.7716 - val_accuracy: 0.2333
Epoch 4/10
26/26 [==============================] - 44s 2s/step - loss: 1.4677 - accuracy: 0.4024 - val_loss: 1.7953 - val_accuracy: 0.2500
Epoch 5/10
26/26 [==============================] - 41s 2s/step - loss: 1.4088 - accuracy: 0.4436 - val_loss: 1.6668 - val_accuracy: 0.3167
Epoch 6/10
26/26 [==============================] - 41s 2s/step - loss: 1.4102 - accuracy: 0.4570 - val_loss: 1.3738 - val_accuracy: 0.5000
Epoch 7/10
26/26 [==============================] - 44s 2s/step - loss: 1.3704 - accuracy: 0.4667 - val_loss: 1.3376 - val_accuracy: 0.5667
Epoch 8/10
26/26 [

In [ ]:
test_loss, test_acc = model.evaluate(train_generator)
print('Test accuracy:', test_acc)

26/26 [==============================] - 28s 1s/step - loss: 1.2654 - accuracy: 0.4982
Test accuracy: 0.4981818199157715


In [ ]:
import os

# Define paths to save the model and labels
# model_path = 'path_to_save_model/my_model'
# label_path = 'path_to_save_labels/labels.txt'

# Save the model
model.save("model1.h5")

# Save the class names (labels) to a text file
class_names = list(train_generator.class_indices.keys())
with open("labels.txt", 'w') as f:
    for class_name in class_names:
        f.write(class_name + '\n')

print("Model and labels saved successfully.")


Model and labels saved successfully.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model_path = '/content/model1.h5'
model = load_model(model_path)

# Load the label file
label_path = '/content/labels.txt'
with open(label_path, 'r') as f:
    class_names = f.read().splitlines()

# Function to preprocess an image
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to classify an image
def classify_image(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]
    return predicted_class_name

# Test the model on a sample image
sample_image_path = '/content/damage3.jpg'
predicted_class = classify_image(sample_image_path)
print('Predicted class:', predicted_class)


1/1 [==============================] - 0s 81ms/step
Predicted class: Physical-Damage
